## Strategy Based on Indicator

In [1]:
import os

os.chdir("..")

In [2]:
from datetime import date
from pytrade import Trade, Strategy, History, Backtest
from pytrade.assets import reset, Stock, Cash, FxRate, Portfolio
from pytrade.events.yahoo import load_yahoo_prices
from pytrade.compliance import Compliance, UnitLimit

In [3]:
# create your stock and portfolio
reset()
spy = Stock("SPY", currency_code="USD")
aud = Cash("AUD")
usd = Cash("USD")
audusd = FxRate("AUDUSD")
portfolio = Portfolio("AUD")
starting_value = int(1e6)
portfolio.transfer(aud, starting_value)

# impose a compliance rule so we are unable to
# hold more than 1000 shares.
portfolio.compliance = Compliance().add_rule(
    UnitLimit(spy, 1000)
)


# define a strategy to buy or sell 100 shares of SPY based on VIX levels
# if we are short USD then also fund this shortfall with AUD
class BuySpyWithIndicator(Strategy):
    def generate_trades(self):
        trades = list()
        spy_holding = portfolio.get_holding_units("SPY")
        usd_holding = portfolio.get_holding_units("USD")
        vix_level = backtest.get_indicator("^VIX")

        if usd_holding < 0:  # then cover this short USD position
            trades.append(Trade(portfolio, usd, int(-usd_holding)+1))

        if vix_level >= 30:
            # buy 100 shares of spy subject to compliance limits
            trades.append(Trade(portfolio, spy, 100))

        if vix_level <= 20 and spy_holding > 0:
            # sell current holdings
            trades.append(Trade(portfolio, spy, -100))

        return trades


# create your backtest instance
backtest = Backtest(BuySpyWithIndicator())
history = History(
    portfolios=portfolio,
    backtest=backtest,
)

# load price events from yahoo for spy, audusd, vix
start_date = date(2015, 10, 1)
end_date = date(2020, 10, 1)
load_yahoo_prices(
    [spy, audusd, "^VIX"],
    backtest,
    start_date=start_date,
    end_date=end_date,
)

# run the backtest and check pnl
backtest.run()

In [4]:
df = history.get()

df

,AUD,AUDUSD,Portfolio,Portfolio_AUD,Portfolio_SPY,Portfolio_USD,SPY,USD,^VIX
2015-10-01,1.0,0.703680,1.000000e+06,1000000.000000,0.0,0.000000,174.149231,1.0,22.549999
2015-10-02,1.0,0.703680,1.000000e+06,1000000.000000,0.0,0.000000,176.750641,1.0,20.940001
2015-10-04,1.0,0.704821,1.000000e+06,1000000.000000,0.0,0.000000,176.750641,1.0,20.940001
2015-10-05,1.0,0.708818,1.000000e+06,1000000.000000,0.0,0.000000,179.895889,1.0,19.540001
2015-10-06,1.0,0.715922,1.000000e+06,1000000.000000,0.0,0.000000,179.279541,1.0,19.400000
...,...,...,...,...,...,...,...,...,...
2020-09-27,1.0,0.704191,1.054886e+06,588065.933307,1000.0,0.189087,328.730011,1.0,26.379999
2020-09-28,1.0,0.709522,1.059074e+06,588065.933307,1000.0,0.189087,334.190002,1.0,26.190001
2020-09-29,1.0,0.714362,1.053334e+06,588065.933307,1000.0,0.189087,332.369995,1.0,26.270000
2020-09-30,1.0,0.716651,1.055365e+06,588065.933307,1000.0,0.189087,334.890015,1.0,26.370001


In [5]:
import cufflinks as cf

columns = ["Portfolio_SPY", "^VIX"]

df[columns].iplot(
    secondary_y="^VIX",
    title="Portfolio Holdings of SPY",
)

In [6]:
columns = ["Portfolio", "^VIX"]

df[columns].iplot(
    secondary_y="^VIX",
    title="Portfolio Holdings of SPY",
)

***